# Simulação para a apresentação

In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
def read_model(modelName):
    with open('modelos/' + modelName, 'rb') as pickle_file:
        content = pickle.load(pickle_file)
        return content

In [3]:
# Importando modelos
svm_small_gp = read_model('model_small_gp.pkl')
svm_md_gp = read_model('model_medium_and_large_gp.pkl')
cluster_model = read_model('cluster_model.pkl')
se = read_model('standardScaler.pkl')

In [4]:
#### Variáveis Utilizadas em cada modelo
gp_medium_features = [
    'limiteEmpresaAnaliseCredito',
    'titulosEmAberto',
    'maiorAtraso',
    'percentualProtestos',
    'valorSolicitado',
    'valorAprovado',
]

gp_small_features = [
    'faturamentoBruto',
    'periodoDemonstrativoEmMeses',
    'limiteEmpresaAnaliseCredito',
    'titulosEmAberto',
    'valorSolicitado',
    'capitalSocial',
    'scorePontualidade'
]

k_means_features = [
    'margemBrutaAcumulada',
    'faturamentoBruto',
    'margemBruta',
    'percentualRisco',
    'intervaloFundacao'
]

In [5]:
def drop_columns_with_many_nulls(df):
    '''
    Colunas com mais de 40% de dados faltantes = drop
    df -> dataframe para ser ajustado
    '''
    percentage_of_nulls = df.isnull().sum() / df.shape[0]
    columns_to_drop = np.array(percentage_of_nulls[percentage_of_nulls > .4].index)
    adjusted_dataframe = df.drop(columns_to_drop, axis=1)
    return adjusted_dataframe

def initial_column_selection(df):
    '''
    ``Helper function``
    Dropa colunas que não serão utilizadas
    '''
    adjusted_df = drop_columns_with_many_nulls(df)
    
    ## Também não quero utilizar datas na minha regressão
    unused_columns = ['primeiraCompra']
    
    # Também não quero colunas que identifiquem o cliente, quero trata-los como um ponto único
    unused_columns = np.concatenate(
        (
            unused_columns, 
            np.array(['numero_solicitacao', 'cnpjSemTraco', 'nomeFantasia', 'razaoSocial', 'status'])
        )
    )
    
    return adjusted_df.drop(unused_columns, axis=1)

In [6]:
from sklearn.preprocessing import StandardScaler

def predict_group(model, df, normalizer, features_used=k_means_features):
    mapped_group_names = {0: 'pequeno', 1: 'medio', 2: 'grande'}
    initial_sample = df.sample(1).copy()
    test_sample = initial_sample[features_used].copy()
    
    total_nulls = test_sample.isnull().sum(axis=1).values[0]
    
    while total_nulls != 0:
        test_sample = test_sample.sample(1).copy()
        total_nulls = test_sample.isnull().sum(axis=1).values[0]
    
    sample_normalized = normalizer.transform(test_sample)
    y_pred = model.predict(sample_normalized)
    
    return (mapped_group_names[y_pred[0]], initial_sample)

In [20]:
def predict_value(model, sample, features_used):
    new_df = pd.DataFrame()
    try:
        initial_sample = sample[1]
        test_sample = initial_sample[features_used].copy()
        features = test_sample.drop('valorAprovado', axis=1)
        y_true = test_sample['valorAprovado']
        log_test_sample = np.log(features + 1)
        y_pred = model.predict(log_test_sample)

        print(f'''
            Valor Real: {y_true.values[0]},
            Valor Predito: {np.exp(y_pred[0])}
        ''')
        return y_pred[0]
    except:
        print('Contém nulos')


## Exemplo de predição

In [8]:
df = pd.read_csv('../data/bootcamp-solicitacoescredito.csv')
df['intervaloFundacao'] = df['intervaloFundacao'].map({'De 0 a 5 anos': 0, 'De 6 a 10 anos': 1, 'De 11 a 16 anos': 2, 'Acima de 17 anos': 3})

In [15]:
df_example = pd.DataFrame()

In [102]:
for i in range(50):
    sample = predict_group(cluster_model, df, se)
    if sample[0] == 'pequeno':
        print('grupo pequeno')
        y_pred = predict_value(svm_small_gp, sample, gp_small_features)
    elif sample[0] == 'medio' or sample[0] == 'grande':
        print(sample[0])
        y_pred = predict_value(svm_md_gp, sample, gp_medium_features)
    sample[1]['y_pred'] = [y_pred]
    
    df_example = pd.concat([df_example, sample[1]])

medio

            Valor Real: 20000.0,
            Valor Predito: 17100.84244564374
        
medio

            Valor Real: 60000.0,
            Valor Predito: 36343.522315581984
        
medio

            Valor Real: 15000.0,
            Valor Predito: 14423.537324648034
        
grupo pequeno
Contém nulos
medio

            Valor Real: 65000.0,
            Valor Predito: 50001.000000000015
        
grupo pequeno
Contém nulos
medio

            Valor Real: 40000.0,
            Valor Predito: 43089.69920337331
        
medio

            Valor Real: 15000.0,
            Valor Predito: 15000.99999999999
        
medio

            Valor Real: 20000.0,
            Valor Predito: 20001.000000000015
        
medio

            Valor Real: 10000.0,
            Valor Predito: 17381.264305067503
        
grupo pequeno
Contém nulos
medio

            Valor Real: 400000.0,
            Valor Predito: 344934.82995318057
        
grupo pequeno
Contém nulos
grupo pequeno
Contém nulos
grupo pequen

KeyboardInterrupt: 

In [106]:
df_example.dropna(subset=['y_pred'], inplace=True)

In [109]:
df_example.to_csv('../predict_examples.csv', index=False)